In [1]:
import importlib

In [2]:
import hnsw
importlib.reload(hnsw)
from hnsw import HNSW
from hnsw import heuristic
import numpy as np
from datasets import load_sift_dataset, calculate_recall

In [3]:
import merge_hnsw
importlib.reload(merge_hnsw)
from merge_hnsw import merge2

In [4]:
distance_count = 0
def l2_distance(a, b):
    global distance_count
    distance_count+=1
    return np.linalg.norm(a - b)

In [ ]:
hnsw_a = HNSW( distance_func=l2_distance, m=5, m0=7, ef=10, ef_construction=30,  neighborhood_construction = heuristic)
hnsw_b = HNSW( distance_func=l2_distance, m=5, m0=7, ef=10, ef_construction=30,  neighborhood_construction = heuristic)

hnsw_a.load('save/sift1m/hnsw_a.txt')
hnsw_b.load('save/sift1m/hnsw_b.txt')

merged_data = hnsw_a.data.copy()
merged_data.update(hnsw_b.data)

In [8]:
%%time
from tqdm.notebook import tqdm
distance_count = 0

jump_ef=20
local_ef=5
next_step_k=3
M=20

hnsw_merged2 = merge2(hnsw_a, hnsw_b, merged_data, jump_ef=jump_ef, local_ef=local_ef, next_step_k=next_step_k, M=M)

Merging level: 0


Merging progress: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000000/1000000 [10:30:29<00:00, 26.43it/s]


Merging level: 1


Merging progress: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 62993/62993 [04:53<00:00, 214.79it/s]


Merging level: 2


...

Merging level: 3


Merging progress: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 272/272 [00:00<00:00, 705.35it/s]


Merging level: 4


Merging progress: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 17/17 [00:00<00:00, 2282.21it/s]

CPU times: user 3h 1min 15s, sys: 26min 17s, total: 3h 27min 33s
Wall time: 10h 35min 41s


In [9]:
hnsw_merged2.save(f'save/sift1m/hnsw_merged2_jef{jump_ef}_lef{local_ef}_nsk{next_step_k}_M{M}.txt')

In [10]:
print(f'Numer of distance calculated during merge: {distance_count}') # 588321997

Numer of distance calculated during merge: 909587415


In [12]:
_, test_data, groundtruth_data = load_sift_dataset(train_file = None,
                                                      test_file='../datasets/sift1m-128d/sift_query.fvecs',
                                                      groundtruth_file='../datasets/sift1m-128d/sift_groundtruth.ivecs')

In [13]:
search_ef = 32
distance_count = 0
recall, _ = calculate_recall(hnsw_merged2, test_data, groundtruth=groundtruth_data, k=5, ef=search_ef)
print(f'{search_ef},{recall}, {distance_count/len(test_data) }') 

Calculating recall...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [01:12<00:00, 137.93it/s]

32,0.8391, 610.5466


In [14]:
search_ef = 40
distance_count = 0
recall, _ = calculate_recall(hnsw_merged2, test_data, groundtruth=groundtruth_data, k=5, ef=search_ef)
print(f'{search_ef},{recall}, {distance_count/len(test_data) }') 

Calculating recall...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [01:38<00:00, 101.12it/s]

40,0.86936, 706.1153


In [15]:
search_ef = 64
distance_count = 0
recall, _ = calculate_recall(hnsw_merged2, test_data, groundtruth=groundtruth_data, k=5, ef=search_ef)
print(f'{search_ef},{recall}, {distance_count/len(test_data) }') 

Calculating recall...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [02:58<00:00, 56.06it/s]

64,0.9195400000000001, 985.9332
